Import lib หลักๆ

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

มาลองดู data กันก่อน

In [3]:
df = pd.read_csv('ai_symptom.csv')
df.head()

,gender,age,summary,search_term
0,male,28,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ"
1,male,27,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล"
2,female,26,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ปวดท้อง
3,male,42,"{""diseases"": [], ""procedures"": [], ""no_symptom...",น้ำมูกไหล
4,female,40,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ตาแห้ง


In [4]:
df.to_csv('testasd.csv')

Data Preparation

summary อยู่ในรูป json, เราจะแยกออกมาเป็น column ต่างๆ

In [5]:
df['summary'] = df['summary'].apply(json.loads) # ทำให้ json data เป็น dict

In [6]:
first_layer_keys = ['diseases', 'procedures', 'no_symptoms', 'idk_symptoms', 'yes_symptoms'] # แยก column ออกมาตาม key
for key in first_layer_keys:
    df[key] = df['summary'].apply(lambda x: x.get(key, []))

In [7]:
df = df.drop('summary', axis=1)

In [8]:
df.head()

,gender,age,search_term,diseases,procedures,no_symptoms,idk_symptoms,yes_symptoms
0,male,28,"มีเสมหะ, ไอ",[],[],[],[],"[{'text': 'เสมหะ', 'answers': ['ลักษณะ เสมหะเป..."
1,male,27,"ไอ, น้ำมูกไหล",[],[],[],[],"[{'text': 'ไอ', 'answers': ['ระยะเวลา 1-3 สัปด..."
2,female,26,ปวดท้อง,[],[],[],[],"[{'text': 'ปวดท้อง', 'answers': ['บริเวณ รอบๆส..."
3,male,42,น้ำมูกไหล,[],[],[],[],"[{'text': 'น้ำมูกไหล', 'answers': ['ระยะเวลา น..."
4,female,40,ตาแห้ง,[],[],[],[],"[{'text': 'ตาแห้ง', 'answers': []}, {'text': '..."


เราจะเห็นเบื้องต้นว่าคอลัมน์ diseases, procedures, no_symptoms, idk_symptoms นั้นเหมือนจะเป็นค่าว่างเป็นส่วนใหญ่ เพราะฉะนั้นเราจะมาสำรวจ data กันก่อน

In [9]:
df['diseases'].describe() #ว่าง

count     1000
unique       1
top         []
freq      1000
Name: diseases, dtype: object

In [10]:
df['procedures'].describe() #ว่าง

count     1000
unique       1
top         []
freq      1000
Name: procedures, dtype: object

In [11]:
df['no_symptoms'].describe()

count     1000
unique      31
top         []
freq       921
Name: no_symptoms, dtype: object

In [12]:
df['idk_symptoms'].describe()

count     1000
unique       8
top         []
freq       980
Name: idk_symptoms, dtype: object

In [13]:
df['yes_symptoms'].describe()

count                                                  1000
unique                                                  591
top       [{'text': 'เจ็บคอ', 'answers': ['บริเวณ ในลำคอ...
freq                                                     39
Name: yes_symptoms, dtype: object

เนื่องจาก diseases, procedures เป็นค่าว่างทั้งหมด เราจะ drop column นี้ออก

In [14]:
df = df.drop(['diseases', 'procedures'], axis=1)

In [15]:
df.head()

,gender,age,search_term,no_symptoms,idk_symptoms,yes_symptoms
0,male,28,"มีเสมหะ, ไอ",[],[],"[{'text': 'เสมหะ', 'answers': ['ลักษณะ เสมหะเป..."
1,male,27,"ไอ, น้ำมูกไหล",[],[],"[{'text': 'ไอ', 'answers': ['ระยะเวลา 1-3 สัปด..."
2,female,26,ปวดท้อง,[],[],"[{'text': 'ปวดท้อง', 'answers': ['บริเวณ รอบๆส..."
3,male,42,น้ำมูกไหล,[],[],"[{'text': 'น้ำมูกไหล', 'answers': ['ระยะเวลา น..."
4,female,40,ตาแห้ง,[],[],"[{'text': 'ตาแห้ง', 'answers': []}, {'text': '..."


นำ value ในคอลัมน์ symptoms ต่างๆมาแตกออก

In [16]:
yes_symptoms_expanded = []

# ลูปไปในทุก row ของ yes_symptoms
for symptoms in df['yes_symptoms']:
    symptoms_dict = {}
    
    # เนื่องจาก data เป็น dict เราสามารถหยิบค่ามาจาก key ได้เลย
    for item in symptoms:
        text = item['text']
        answers = item['answers']
        
        # สร้าง column ใหม่ด้วยค่า text
        symptoms_dict[text] = answers
    
    yes_symptoms_expanded.append(symptoms_dict)

# นำ list ที่ได้มาทำ Dataframe เพื่อเตรียมไว้ในการจัดการต่อไป
yes_symptoms_df = pd.DataFrame(yes_symptoms_expanded)



In [17]:
yes_symptoms_df.head()

,เสมหะ,ไอ,การรักษาก่อนหน้า,น้ำมูกไหล,ปวดท้อง,ตาแห้ง,ปวดกระดูก,เจ็บคอ,อาเจียน,ปวดเมื่อยกล้ามเนื้อ,...,ประวัติไขมันสูง,ปวดบริเวณใบหน้า,itchy nose,จ้ำเลือด,"ประจำเดือนมาน้อย, ประจำเดือนขาด",จุดเลือดออก,blurry vision,fainted,lightheadness,วูบ
0,[ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว],[ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน)],[การรักษาก่อนหน้า ไม่เคย],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"[ระยะเวลา 1-3 สัปดาห์, ลักษณะ ไอไม่มีเสมหะ ไอแ...",[การรักษาก่อนหน้า เคยทานยาเองแล้วไม่ดีขึ้น],"[ระยะเวลา 10 - 90 วัน, ประวัติ ATK ทำแล้ว ได้ผ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,[การรักษาก่อนหน้า ไม่เคย],NaN,"[บริเวณ รอบๆสะดือ, ระยะเวลา ตั้งแต่ 1 วัน ถึง ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,[การรักษาก่อนหน้า ไม่เคย],"[ระยะเวลา น้อยกว่า 10 วัน, ประวัติ ATK ยังไม่ไ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,[การรักษาก่อนหน้า ไม่เคย],NaN,NaN,[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
yes_symptoms_df['การรักษาก่อนหน้า']

0                        [การรักษาก่อนหน้า ไม่เคย]
1      [การรักษาก่อนหน้า เคยทานยาเองแล้วไม่ดีขึ้น]
2                        [การรักษาก่อนหน้า ไม่เคย]
3                        [การรักษาก่อนหน้า ไม่เคย]
4                        [การรักษาก่อนหน้า ไม่เคย]
                          ...                     
995                      [การรักษาก่อนหน้า ไม่เคย]
996                      [การรักษาก่อนหน้า ไม่เคย]
997                      [การรักษาก่อนหน้า ไม่เคย]
998                      [การรักษาก่อนหน้า ไม่เคย]
999                      [การรักษาก่อนหน้า ไม่เคย]
Name: การรักษาก่อนหน้า, Length: 1000, dtype: object

In [19]:
yes_symptoms_df['การรักษาก่อนหน้า'].value_counts()

การรักษาก่อนหน้า
[การรักษาก่อนหน้า ไม่เคย]                      702
[การรักษาก่อนหน้า เคยทานยาเองแล้วไม่ดีขึ้น]    130
[การรักษาก่อนหน้า เคยรักษามาก่อน]               87
Name: count, dtype: int64

In [20]:
# เนื่องจากคอลัมน์ 'การรักษาก่อนหน้า' นั้นมีค่าทุกตัว, แต่ค่านั้นเป็น text บอกว่าเคยได้รับการรักษาหรือไม่
# เพราะฉะนั้นเราจะแทนที่ค่าการไม่เคยด้วย 0 และเคยด้วย 1
yes_symptoms_df['การรักษาก่อนหน้า'] = yes_symptoms_df['การรักษาก่อนหน้า'].apply(
    lambda x: 0 if isinstance(x, list) and 'ไม่เคย' in x[0] else 1 if isinstance(x, list) else 0
)


In [21]:
from googletrans import Translator

# เนื่องจากข้อมูลที่ได้มีทั้งไทยและอังกฤษ เราจะต้องมีการแปลภาษา
async def translate_text(text):
    translator = Translator()
    translated = await translator.translate(text, src='en', dest='th')
    return translated.text

In [22]:
import re

# เราจะ loop ตามชื่อคอลัมน์ และแยกว่าคอลัมน์ไหนเป็นภาษาอังกฤษด้วย regex
english_words = []
thai_words = []
for word in yes_symptoms_df.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)
len(english_words)

49

In [23]:
import difflib

# ฟังชันก์หาคำที่ความหมายใกล้กันด้วยไลบรารี่ difflib
def find_closest_match(word, choices):
    match = difflib.get_close_matches(word, choices, n=1, cutoff=0.5)
    return match[0] if match else None

# แปลคำอังกฤษและมาเทียบกับคำไทยที่มีอยู่
matching_pairs = {}
for eng_word in english_words:
    translated = await translate_text(eng_word)
    closest_thai = find_closest_match(translated, thai_words)
    if closest_thai:
        matching_pairs[eng_word] = closest_thai

# คำที่ความหมายใกล้กัน
matching_pairs

{'Fever': 'เป็นไข้',
 'Previous treatment': 'การรักษาก่อนหน้า',
 'diarrhea': 'ท้องเสีย',
 'abdominal pain': 'ปวดท้อง',
 'straining to urinate': 'ปัสสาวะขุ่น',
 'back pain': 'ปวดหลัง',
 'cough': 'ไอ',
 'wheezing sound': 'เสียงแหบ',
 'Dizzy': 'อาเจียน',
 'shortness of breath': 'หายใจหอบเหนื่อย',
 'itch': 'คัน',
 'skin rash': 'ก้อนที่ผิวหนัง',
 'hearing loss': 'การได้ยินลดลง',
 'Headache': 'ปวดกระดูก',
 'sore throat': 'เจ็บคอ',
 'vomiting': 'อาเจียน',
 'dry skin': 'ผิวแห้ง',
 'ear pain': 'ปวดหู',
 'foot pain': 'ปวดเท้า',
 'joint pain': 'ปวดข้อ',
 'runny nose': 'น้ำมูกไหล',
 'dry throat': 'คอแห้ง',
 'Blood in urine': 'ปัสสาวะขุ่น',
 'stuffy nose': 'ปวดจมูก',
 'shortness of breath when lying down': 'หายใจหอบเหนื่อย',
 'phlegm': 'เสมหะ',
 'History of trauma': 'ประวัติความดันสูง',
 'eye irritation': 'เคืองตา',
 'History of hypertension (high blood pressure)': 'ประวัติความดันสูง',
 'nausea': 'คลื่นไส้',
 'neck pain': 'ปวดต้นคอ',
 'ankle pain': 'ปวดข้อเท้า',
 'itchy nose': 'คัน',
 'fainted': 'เ

In [24]:
# นำคอลัมน์ไทยกับอังกฤษที่ความหมายเหมือนกันมารวมกัน
for eng_col, thai_col in matching_pairs.items():
    yes_symptoms_df[thai_col] = yes_symptoms_df[thai_col].combine_first(yes_symptoms_df[eng_col])
    yes_symptoms_df.drop(columns=[eng_col], inplace=True)

/tmp/ipykernel_189736/4185747457.py:3: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  yes_symptoms_df[thai_col] = yes_symptoms_df[thai_col].combine_first(yes_symptoms_df[eng_col])


In [25]:
yes_symptoms_df.head()

,เสมหะ,ไอ,การรักษาก่อนหน้า,น้ำมูกไหล,ปวดท้อง,ตาแห้ง,ปวดกระดูก,เจ็บคอ,อาเจียน,ปวดเมื่อยกล้ามเนื้อ,...,ปวดกราม,genitals itching,narrow stool,ประวัติไขมันสูง,ปวดบริเวณใบหน้า,จ้ำเลือด,"ประจำเดือนมาน้อย, ประจำเดือนขาด",จุดเลือดออก,blurry vision,วูบ
0,[ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว],[ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน)],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"[ระยะเวลา 1-3 สัปดาห์, ลักษณะ ไอไม่มีเสมหะ ไอแ...",1,"[ระยะเวลา 10 - 90 วัน, ประวัติ ATK ทำแล้ว ได้ผ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0,NaN,"[บริเวณ รอบๆสะดือ, ระยะเวลา ตั้งแต่ 1 วัน ถึง ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0,"[ระยะเวลา น้อยกว่า 10 วัน, ประวัติ ATK ยังไม่ไ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0,NaN,NaN,[],NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# อาจมีคำอังกฤษบางคำที่ google translate แปลแล้วไม่มีความใกล้เคียงกับคำไทยคำไหนเลย
# เราจะทิ้งข้อมูลส่วนนั้นไปก่อนโอกาสนี้
english_words = []
thai_words = []
for word in yes_symptoms_df.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)
len(english_words)

14

In [27]:
english_words

['axillary mass',
 'acne',
 'Jaundice',
 'eye mucus, eye discharge',
 'ear discharge/drainage',
 'unsteady, loss of balance',
 'Animal related injury',
 'fatigue associated with exertion',
 'slurred speech',
 'Weight loss',
 'Loss of appetite',
 'genitals itching',
 'narrow stool',
 'blurry vision']

In [28]:
yes_symptoms_df = yes_symptoms_df.drop(english_words, axis=1)

In [29]:
# แยก column ของ no_symptoms ออกมาเหมือนกัน
no_symptoms_expanded = []

for symptoms in df['no_symptoms']:
    symptoms_dict = {}
    
    for item in symptoms:
        text = item['text']
        answers = item['answers']
        symptoms_dict[text] = answers
    
    no_symptoms_expanded.append(symptoms_dict)

no_symptoms_df = pd.DataFrame(no_symptoms_expanded)



In [30]:
# แยกภาษา
english_words = []
thai_words = []
for word in no_symptoms_df.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)


In [31]:
#จับคู่
matching_pairs = {}
for eng_word in english_words:
    translated = await translate_text(eng_word)
    closest_thai = find_closest_match(translated, thai_words)
    if closest_thai:
        matching_pairs[eng_word] = closest_thai

matching_pairs

{'fainted': 'เป็นไข้',
 'History of hypertension (high blood pressure)': 'ประวัติความดันสูง',
 'History of diabetic': 'ประวัติเบาหวาน',
 'History of hypercholesterolemia (high cholesterol)': 'ประวัติไขมันสูง',
 'History of cardiac disease': 'ประวัติโรคหัวใจ',
 'infrequent menstrual cycle, inconsistent menstrual cycle': 'ประจำเดือนมากกว่าปกติ',
 'history of gastritis': 'ประวัติโรคกระเพาะ'}

In [32]:
# รวมคอลัมน์
for eng_col, thai_col in matching_pairs.items():
    no_symptoms_df[thai_col] = no_symptoms_df[thai_col].combine_first(no_symptoms_df[eng_col])  # Fill NaN with matching column
    no_symptoms_df.drop(columns=[eng_col], inplace=True)

In [33]:
# drop คอลัมน์อังกฤษที่เหลือ
english_words = []
thai_words = []
for word in no_symptoms_df.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)

no_symptoms_df = no_symptoms_df.drop(english_words, axis=1)
no_symptoms_df

,ปวดท้อง,"ประจำเดือนมาน้อย, ประจำเดือนขาด",ประวัติความดันสูง,ประวัติเบาหวาน,ประวัติไขมันสูง,ประวัติโรคหัวใจ,ท้องผูก,เป็นไข้,ประวัติโรคกระเพาะ,เดินเซ ทรงตัวไม่ได้,วูบ,แขนขาอ่อนแรง,แผล,คันอวัยวะเพศ,ประจำเดือนมากกว่าปกติ,ตกขาวผิดปกติ,หายใจหอบเหนื่อย
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# แยก column ของ idk_symptoms ออกมาเหมือนกัน
idk_symptoms_expanded = []

for symptoms in df['idk_symptoms']:
    symptoms_dict = {}
    
    for item in symptoms:
        text = item['text']
        answers = item['answers']
        
        symptoms_dict[text] = answers
    
    idk_symptoms_expanded.append(symptoms_dict)

idk_symptoms_df = pd.DataFrame(idk_symptoms_expanded)


In [35]:
# แยกภาษา
english_words = []
thai_words = []
for word in idk_symptoms_df.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)
len(english_words)


1

In [36]:
#จับคู่
matching_pairs = {}
for eng_word in english_words:
    translated = await translate_text(eng_word)
    closest_thai = find_closest_match(translated, thai_words)
    if closest_thai:
        matching_pairs[eng_word] = closest_thai

matching_pairs

{'History of hypertension (high blood pressure)': 'ประวัติความดันสูง'}

In [37]:
# รวมคอลัมน์
for eng_col, thai_col in matching_pairs.items():
    idk_symptoms_df[thai_col] = idk_symptoms_df[thai_col].combine_first(idk_symptoms_df[eng_col])
    idk_symptoms_df.drop(columns=[eng_col], inplace=True)

In [38]:
# drop คอลัมน์อังกฤษที่เหลือ
english_words = []
thai_words = []
for word in idk_symptoms_df.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)
        
idk_symptoms_df = idk_symptoms_df.drop(english_words, axis=1)
idk_symptoms_df

,ประวัติความดันสูง,ประวัติโรคกระเพาะ,ประวัติเบาหวาน,ประวัติไขมันสูง,ประวัติโรคหัวใจ,ประวัติไตวายระยะสุดท้าย
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,[],[]
996,NaN,NaN,NaN,NaN,NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN
998,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df

,gender,age,search_term,no_symptoms,idk_symptoms,yes_symptoms
0,male,28,"มีเสมหะ, ไอ",[],[],"[{'text': 'เสมหะ', 'answers': ['ลักษณะ เสมหะเป..."
1,male,27,"ไอ, น้ำมูกไหล",[],[],"[{'text': 'ไอ', 'answers': ['ระยะเวลา 1-3 สัปด..."
2,female,26,ปวดท้อง,[],[],"[{'text': 'ปวดท้อง', 'answers': ['บริเวณ รอบๆส..."
3,male,42,น้ำมูกไหล,[],[],"[{'text': 'น้ำมูกไหล', 'answers': ['ระยะเวลา น..."
4,female,40,ตาแห้ง,[],[],"[{'text': 'ตาแห้ง', 'answers': []}, {'text': '..."
...,...,...,...,...,...,...
995,male,38,"บวม, ปวดข้อ","[{'text': 'แผล', 'answers': []}]","[{'text': 'ประวัติไตวายระยะสุดท้าย', 'answers'...","[{'text': 'บวมตามร่างกาย', 'answers': ['ตำแหน่..."
996,male,33,เจ็บคอ,[],[],"[{'text': 'เจ็บคอ', 'answers': ['บริเวณ ในลำคอ..."
997,male,45,ไข้,[],[],"[{'text': 'เป็นไข้', 'answers': ['ระยะเวลา 1-3..."
998,female,73,ไข้,[],[],"[{'text': 'เป็นไข้', 'answers': ['ระยะเวลา 1-3..."


In [40]:
# ทำ one hot encoding ให้คอลัมน์ search_term ด้วย pd.get_dummies
search_term = df['search_term'].str.get_dummies(sep=', ')

In [41]:
search_term

,Animal bite,Blurry vision,Decreased stool caliber,Dizzy,DizzyBlack out,Dry throat,Ear discharge,Ear dischargeEar pain,Ear pain,Eye discharge,...,ไอ,ไอกลางคืน,ไอกลางคืนมีเสมหะ,ไอคัดจมูก,ไอน้ำมูกไหล,ไอผื่น,ไอมีเสมหะ,ไอเจ็บคอ,ไอเป็นเลือด,ไอไอกลางคืน
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# search_term มีภาษาอังกฤษ เพราะฉะนั้นเราจะใช้วิธีจับคู่คำเหมือนเดิม
english_words = []
thai_words = []
for word in search_term.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)
len(english_words)

60

In [43]:
# จับคู่
matching_pairs = {}
for eng_word in english_words:
    translated = await translate_text(eng_word)
    closest_thai = find_closest_match(translated, thai_words)
    if closest_thai:
        matching_pairs[eng_word] = closest_thai

matching_pairs

{'Decreased stool caliber': 'อุจจาระลำเล็กลง',
 'Dizzy': 'เวียนศีรษะ',
 'Dry throat': 'คอแห้ง',
 'Ear dischargeEar pain': 'ปวดหู',
 'Ear pain': 'ปวดหู',
 'Eye irritation': 'เคืองตา',
 'Eye pain': 'ปวดตา',
 'Fever': 'ไข้',
 'Headache': 'เวียนศีรษะ',
 'Headachecough': 'ปวดหัว',
 'Hearing loss': 'การได้ยินลดลง',
 'Hearing lossEar discharge': 'การได้ยินลดลง',
 'Hoarseness': 'เสียงแหบ',
 'Runny nose': 'คันจมูกน้ำมูกไหล',
 'Sore throat': 'เจ็บคอ',
 'Sore throatStuffy nose': 'ไอเจ็บคอ',
 'Stuffy nose': 'จมูกตัน',
 'Yellowish skin': 'ตาเหลือง',
 'ankle pain': 'ปวดข้อเท้า',
 'armpit lump': 'ก้อนที่รักแร้',
 'back pain': 'ปวดหลัง',
 'bloody urine': 'ปัสสาวะบ่อย',
 'breathless on lying': 'หายใจไม่ออก',
 'breathless on lyingStuffy nose': 'หายใจไม่ออก',
 'cough': 'ไอ',
 'coughLightheaded': 'ปวดเข่า',
 'diarrhea': 'ท้องเสีย',
 'dry skin': 'ผิวแห้ง',
 'foot pain': 'ปวดเท้า',
 'itch': 'คัน',
 'itchy nosesneezing': 'จมูกตัน',
 'joint pain': 'ปวดข้อ',
 'labored breathing': 'หายใจไม่ออก',
 'nasal congest

In [44]:
# รวมคอลัมน์
for eng_col, thai_col in matching_pairs.items():
    search_term[thai_col] = search_term[thai_col].combine_first(search_term[eng_col])  # Fill NaN with matching column
    search_term.drop(columns=[eng_col], inplace=True)

In [45]:
# drop คอลัมน์อังกฤษที่เหลือ
english_words = []
thai_words = []
for word in search_term.columns:
    if re.match(r'[a-zA-Z]', word):
        english_words.append(word)
    else:
        thai_words.append(word)

search_term = search_term.drop(english_words, axis=1)
search_term

,กระแทก,กลืนติด,กลืนลำบาก,กลืนเจ็บ,การได้ยินลดลง,ก้อนที่ขา,ก้อนที่ผิวหนัง,ก้อนที่รักแร้,ก้อนที่ศีรษะ,ก้อนที่หลังหู,...,ไอ,ไอกลางคืน,ไอกลางคืนมีเสมหะ,ไอคัดจมูก,ไอน้ำมูกไหล,ไอผื่น,ไอมีเสมหะ,ไอเจ็บคอ,ไอเป็นเลือด,ไอไอกลางคืน
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


เนื่องจาก search term มีความเป็นไปได้มากมายตามอาการ user ที่พิมพ์เข้ามา
เพราะฉะนั้นเราจะจำแนกกลุ่มตามอาการเพื่อความสะดวก

In [46]:
from transformers import pipeline

# # จำแนกกลุ่มหลักๆตามอาการที่มี
# categories = [
#     "อาการทางระบบหายใจ",  
#     "อาการทางระบบทางเดินอาหาร",  
#     "อาการทางระบบประสาท",  
#     "อาการทางผิวหนังและภูมิแพ้",  
#     "อาการปวดและการอักเสบ",  
#     "อาการไข้และการติดเชื้อ", 
#     "อาการทางระบบหัวใจและหลอดเลือด",
#     "อาการทางระบบปัสสาวะ",  
#     "อาการทางสุขภาพจิต",
#     "อาการทั่วไปอื่นๆ"  
# ]

# # เราจะใช้ lib transformer ของ hugging face จำแนกอาการลงตามหมวดหมู่
# classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

# # ทำการจำแนก column
# results = classifier(list(search_term.columns), candidate_labels=categories, multi_label=True)

# # นำ result ที่ได้มา process เพื่อนำค่าไปใส่ในหมวดหมู่ที่ความใกล้เคียงกับอาการ
# category_mapping = {}
# for i, col in enumerate(search_term.columns):
#     matched_categories = [categories[j] for j, score in enumerate(results[i]["scores"]) if score > 0.8]
#     category_mapping[col] = matched_categories if matched_categories else ["อาการทั่วไปอื่นๆ"]  # หากไม่ตรงกับหมวดหมู่ไหนเลยก็จะใช้ อาการทั่วไปอื่นๆ เป็นค่า default

# # นำค่าที่ได้มา group กันตามหมวดหมู่
# category_groups = {}
# for col, categories in category_mapping.items():
#     for category in categories:
#         category_groups.setdefault(category, []).append(col)

# # สร้าง Dataframe มาเพื่อเก็บค่า search term จามหมวดหมู่
# search_term_df = pd.DataFrame({cat: search_term[cols].sum(axis=1) for cat, cols in category_groups.items() if cols})


# search_term_df['อาการทั่วไปอื่นๆ'] = 0

/home/woreinor/miniconda3/envs/tf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-30 19:07:07.025248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743336427.039972  189736 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743336427.044786  189736 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743336427.060667  189736 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than 

In [47]:
# เนื่องจากการจำแนกใช้เวลาในการรันนาน สามารถใช้ผลที่ผมรันและเซฟมาแล้วเพื่อเป็นการประหยัดเวลาได้
search_term_df = pd.read_csv('search_term.csv')

เตรียมการเรียบร้อย ถีงเวลาประกอบร่าง data

In [48]:
# เพิ่ม column no_symptoms ที่แยกแล้วลงไปและ drop อันเก่าทิ้ง
no_symptoms_df = no_symptoms_df.add_prefix("no_symptoms_")  
df = pd.concat([df, no_symptoms_df], axis=1)  
df = df.drop('no_symptoms', axis=1)
df.head()

,gender,age,search_term,idk_symptoms,yes_symptoms,no_symptoms_ปวดท้อง,"no_symptoms_ประจำเดือนมาน้อย, ประจำเดือนขาด",no_symptoms_ประวัติความดันสูง,no_symptoms_ประวัติเบาหวาน,no_symptoms_ประวัติไขมันสูง,...,no_symptoms_เป็นไข้,no_symptoms_ประวัติโรคกระเพาะ,no_symptoms_เดินเซ ทรงตัวไม่ได้,no_symptoms_วูบ,no_symptoms_แขนขาอ่อนแรง,no_symptoms_แผล,no_symptoms_คันอวัยวะเพศ,no_symptoms_ประจำเดือนมากกว่าปกติ,no_symptoms_ตกขาวผิดปกติ,no_symptoms_หายใจหอบเหนื่อย
0,male,28,"มีเสมหะ, ไอ",[],"[{'text': 'เสมหะ', 'answers': ['ลักษณะ เสมหะเป...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,male,27,"ไอ, น้ำมูกไหล",[],"[{'text': 'ไอ', 'answers': ['ระยะเวลา 1-3 สัปด...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,female,26,ปวดท้อง,[],"[{'text': 'ปวดท้อง', 'answers': ['บริเวณ รอบๆส...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male,42,น้ำมูกไหล,[],"[{'text': 'น้ำมูกไหล', 'answers': ['ระยะเวลา น...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,female,40,ตาแห้ง,[],"[{'text': 'ตาแห้ง', 'answers': []}, {'text': '...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# เพิ่ม column idk_symptoms ที่แยกแล้วลงไปและ drop อันเก่าทิ้ง
idk_symptoms_df = idk_symptoms_df.add_prefix("idk_symptoms_")
df = pd.concat([df, idk_symptoms_df], axis=1)
df = df.drop('idk_symptoms', axis=1)
df.head()

,gender,age,search_term,yes_symptoms,no_symptoms_ปวดท้อง,"no_symptoms_ประจำเดือนมาน้อย, ประจำเดือนขาด",no_symptoms_ประวัติความดันสูง,no_symptoms_ประวัติเบาหวาน,no_symptoms_ประวัติไขมันสูง,no_symptoms_ประวัติโรคหัวใจ,...,no_symptoms_คันอวัยวะเพศ,no_symptoms_ประจำเดือนมากกว่าปกติ,no_symptoms_ตกขาวผิดปกติ,no_symptoms_หายใจหอบเหนื่อย,idk_symptoms_ประวัติความดันสูง,idk_symptoms_ประวัติโรคกระเพาะ,idk_symptoms_ประวัติเบาหวาน,idk_symptoms_ประวัติไขมันสูง,idk_symptoms_ประวัติโรคหัวใจ,idk_symptoms_ประวัติไตวายระยะสุดท้าย
0,male,28,"มีเสมหะ, ไอ","[{'text': 'เสมหะ', 'answers': ['ลักษณะ เสมหะเป...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,male,27,"ไอ, น้ำมูกไหล","[{'text': 'ไอ', 'answers': ['ระยะเวลา 1-3 สัปด...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,female,26,ปวดท้อง,"[{'text': 'ปวดท้อง', 'answers': ['บริเวณ รอบๆส...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male,42,น้ำมูกไหล,"[{'text': 'น้ำมูกไหล', 'answers': ['ระยะเวลา น...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,female,40,ตาแห้ง,"[{'text': 'ตาแห้ง', 'answers': []}, {'text': '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# เพิ่ม column yes_symptoms ที่แยกแล้วลงไปและ drop อันเก่าทิ้ง
yes_symptoms_df = yes_symptoms_df.add_prefix("yes_symptoms_")
df = pd.concat([df, yes_symptoms_df], axis=1)
df = df.drop('yes_symptoms', axis=1)
df.head()

,gender,age,search_term,no_symptoms_ปวดท้อง,"no_symptoms_ประจำเดือนมาน้อย, ประจำเดือนขาด",no_symptoms_ประวัติความดันสูง,no_symptoms_ประวัติเบาหวาน,no_symptoms_ประวัติไขมันสูง,no_symptoms_ประวัติโรคหัวใจ,no_symptoms_ท้องผูก,...,yes_symptoms_ก้อนบริเวณขาหนีบ,yes_symptoms_ไอเป็นเลือด,yes_symptoms_เลือดกำเดาไหล,yes_symptoms_ปวดกราม,yes_symptoms_ประวัติไขมันสูง,yes_symptoms_ปวดบริเวณใบหน้า,yes_symptoms_จ้ำเลือด,"yes_symptoms_ประจำเดือนมาน้อย, ประจำเดือนขาด",yes_symptoms_จุดเลือดออก,yes_symptoms_วูบ
0,male,28,"มีเสมหะ, ไอ",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,male,27,"ไอ, น้ำมูกไหล",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,female,26,ปวดท้อง,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male,42,น้ำมูกไหล,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,female,40,ตาแห้ง,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# เพิ่ม column search_term ที่แยกแล้วลงไปและ drop อันเก่าทิ้ง
search_term_df = search_term_df.add_prefix("search_term_df")
df = pd.concat([df, search_term_df], axis=1)
df = df.drop('search_term', axis=1)
df.head()

,gender,age,no_symptoms_ปวดท้อง,"no_symptoms_ประจำเดือนมาน้อย, ประจำเดือนขาด",no_symptoms_ประวัติความดันสูง,no_symptoms_ประวัติเบาหวาน,no_symptoms_ประวัติไขมันสูง,no_symptoms_ประวัติโรคหัวใจ,no_symptoms_ท้องผูก,no_symptoms_เป็นไข้,...,search_term_dfsearch_term_อาการทางระบบหายใจ,search_term_dfsearch_term_อาการทางระบบทางเดินอาหาร,search_term_dfsearch_term_อาการทางระบบประสาท,search_term_dfsearch_term_อาการทางผิวหนังและภูมิแพ้,search_term_dfsearch_term_อาการปวดและการอักเสบ,search_term_dfsearch_term_อาการไข้และการติดเชื้อ,search_term_dfsearch_term_อาการทางระบบหัวใจและหลอดเลือด,search_term_dfsearch_term_อาการทางระบบปัสสาวะ,search_term_dfsearch_term_อาการทางสุขภาพจิต,search_term_dfsearch_term_อาการทั่วไปอื่นๆ
0,male,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,2,2,2,2,2,1,0,0
1,male,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,2,2,2,1,1,1,0,0
2,female,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,0,0,0
3,male,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,0,0,0,0,0
4,female,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,1,0,0,0,0,0


In [52]:
temp_prev = df['yes_symptoms_การรักษาก่อนหน้า'].copy()

In [53]:
temp_prev

0      0
1      1
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: yes_symptoms_การรักษาก่อนหน้า, Length: 1000, dtype: int64

In [54]:
# เนื่องจาก value ใน Dataframe ส่วนใหญ่เป็น NaN เราจะทำการแทนค่านั้นๆด้วย 0 เพื่อการทำโมเดล
df.iloc[:, 2:] = df.iloc[:, 2:].applymap(
    lambda x: 1 if isinstance(x, list) else (1 if pd.notna(x) else 0)
)

/tmp/ipykernel_189736/821330626.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.iloc[:, 2:] = df.iloc[:, 2:].applymap(


In [55]:
df['yes_symptoms_การรักษาก่อนหน้า']  =  temp_prev

In [56]:
df['yes_symptoms_การรักษาก่อนหน้า']

0      0
1      1
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: yes_symptoms_การรักษาก่อนหน้า, Length: 1000, dtype: int64

In [57]:
# เนื่องจากเพศเป็น str เราจะใช้ get_dummies เพื่อทำให้เป็น int
dummies = pd.get_dummies(df['gender'],drop_first=True,dtype='int')

df = pd.concat([df.drop('gender',axis=1),dummies],axis=1)

In [58]:
df.head()

,age,no_symptoms_ปวดท้อง,"no_symptoms_ประจำเดือนมาน้อย, ประจำเดือนขาด",no_symptoms_ประวัติความดันสูง,no_symptoms_ประวัติเบาหวาน,no_symptoms_ประวัติไขมันสูง,no_symptoms_ประวัติโรคหัวใจ,no_symptoms_ท้องผูก,no_symptoms_เป็นไข้,no_symptoms_ประวัติโรคกระเพาะ,...,search_term_dfsearch_term_อาการทางระบบทางเดินอาหาร,search_term_dfsearch_term_อาการทางระบบประสาท,search_term_dfsearch_term_อาการทางผิวหนังและภูมิแพ้,search_term_dfsearch_term_อาการปวดและการอักเสบ,search_term_dfsearch_term_อาการไข้และการติดเชื้อ,search_term_dfsearch_term_อาการทางระบบหัวใจและหลอดเลือด,search_term_dfsearch_term_อาการทางระบบปัสสาวะ,search_term_dfsearch_term_อาการทางสุขภาพจิต,search_term_dfsearch_term_อาการทั่วไปอื่นๆ,male
0,28,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,27,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
2,26,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,42,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,40,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [59]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# เนื่องจากอายุเป็นเลข 2 หลัก เราจะทำการ normalize ด้วย min-max scaler
scaler = MinMaxScaler()
df['age'] = scaler.fit_transform(df[['age']])

In [60]:
column_df = pd.DataFrame([df.iloc[0]])

In [61]:
# save ตัว scaler เก็บไว้เพื่อใช้ในการ scale ข้อมูลที่จะใช้เป็น input
joblib.dump(scaler, 'age_scaler.pkl')

['age_scaler.pkl']

In [62]:
age_scaler = joblib.load('age_scaler.pkl')

In [63]:
#เตรียม data เพื่อใช้ในการเทส
input_df = pd.read_csv('input.csv')
input_df = input_df.drop(['Unnamed: 0', 'search_term_อาการทางระบบหายใจ'],axis=1)
input_df['age'] = 30
input_df

,age,yes_symptoms_เสมหะ,yes_symptoms_ไอ,yes_symptoms_การรักษาก่อนหน้า,search_term_อาการทางระบบทางเดินอาหาร,search_term_อาการทางระบบประสาท,search_term_อาการทางผิวหนังและภูมิแพ้,search_term_อาการปวดและการอักเสบ,search_term_อาการไข้และการติดเชื้อ,search_term_อาการทางระบบหัวใจและหลอดเลือด,search_term_อาการทางระบบปัสสาวะ,search_term_อาการทางสุขภาพจิต,search_term_อาการทั่วไปอื่นๆ,male
0,30,1,1,1,1,1,1,1,1,1,1,1,1,1


Model building

วันนี้เราจะทำ Similarity-Based Recommendation

In [64]:
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

# เตรียม data เพื่อใช้ในการทำโมเดล
feature_columns = ["age", "male"] 
symptom_columns = [col for col in column_df.columns if col.startswith("no_symptoms_") or 
                    col.startswith("idk_symptoms_") or 
                    col.startswith("yes_symptoms_")]
search_columns = [col for col in column_df.columns if col.startswith("search_term_")]

all_columns = feature_columns + search_columns + symptom_columns
user_feature_matrix = column_df[all_columns].copy()

# ทำให้ data ทุกท่องเป็นตัวเลข
user_feature_matrix = user_feature_matrix.apply(pd.to_numeric, errors='coerce')

# data มาทำให้เป็น sparse matrix เพื่อใช้เทรนโมเดล
sparse_matrix = csr_matrix(user_feature_matrix.values)


In [65]:
# ประกาศโมเดลและเทรน
svd = TruncatedSVD(n_components=3, random_state=42)
svd.fit(sparse_matrix)

TruncatedSVD(n_components=3, random_state=42)

In [66]:
joblib.dump(svd, "svd_model.pkl")

['svd_model.pkl']

In [67]:
# ทำฟังชั่นการ recommend อาการที่เป็นไปได้ต่อไปจาก input ของ user
def recommend_symptoms(svd, user_vector, all_columns, top_n=3):
    user_vector = np.array(user_vector).reshape(1, -1)
    predicted_scores = svd.transform(user_vector) @ svd.components_
    scores_dict = dict(zip(all_columns, predicted_scores.flatten()))
    sorted_features = sorted(scores_dict, key=scores_dict.get, reverse=True)
    return [feature for feature in sorted_features if "symptoms_" in feature][:top_n]

In [68]:
# ทำให้อินพุตจาก user อยู่ในรูปแบบที่โมเดลใช้ได้
def process_user_input(user_data, all_columns):
    user_vector = np.zeros(len(all_columns), dtype=float)
    
    for i, feature in enumerate(all_columns):
        if feature in user_data:
            user_vector[i] = user_data[feature]
    
    return user_vector

In [69]:
# เรียกใช้เพื่อ process input และ recommend symptoms
user_vector = process_user_input(input_df, all_columns)
recommendations = recommend_symptoms(svd, user_vector, all_columns) 

/tmp/ipykernel_189736/1233716034.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  user_vector[i] = user_data[feature]


In [70]:
print("Recommended Symptoms:", recommendations)

Recommended Symptoms: ['yes_symptoms_เสมหะ', 'yes_symptoms_ไอ', 'no_symptoms_ปวดท้อง']


API

In [71]:
# ใช้ quart เพื่อเรียก async
from quart import Quart, request, jsonify

In [ ]:
app = Quart(__name__)

@app.route('/recommend', methods=['POST'])
async def recommend():
    user_data = request.json
    #turn json to df
    csv_data = []
    
    for entry in user_data:
        csv_entry = {
            "gender": entry["gender"],
            "age": entry["age"],
            "summary": json.dumps(entry["summary"], ensure_ascii=False),  # Convert to JSON string
            "search_term": ", ".join(entry["search_term"])  # Convert list to comma-separated string
        }
        csv_data.append(csv_entry)
    
    # Convert to DataFrame and save as CSV
    df = pd.DataFrame(csv_data)
    df['summary'] = df['summary'].apply(json.loads) # ทำให้ json data เป็น dict
    first_layer_keys = ['diseases', 'procedures', 'no_symptoms', 'idk_symptoms', 'yes_symptoms'] # แยก column ออกมาตาม key
    for key in first_layer_keys:
        df[key] = df['summary'].apply(lambda x: x.get(key, []))
    df = df.drop(['summary','diseases', 'procedures'], axis=1)
    yes_symptoms_expanded = []

    # ลูปไปในทุก row ของ yes_symptoms
    for symptoms in df['yes_symptoms']:
        symptoms_dict = {}
        
        # เนื่องจาก data เป็น dict เราสามารถหยิบค่ามาจาก key ได้เลย
        for item in symptoms:
            text = item['text']
            answers = item['answers']
            
            # สร้าง column ใหม่ด้วยค่า text
            symptoms_dict[text] = answers
        
        yes_symptoms_expanded.append(symptoms_dict)

    # นำ list ที่ได้มาทำ Dataframe เพื่อเตรียมไว้ในการจัดการต่อไป
    yes_symptoms_df = pd.DataFrame(yes_symptoms_expanded)
    yes_symptoms_df['การรักษาก่อนหน้า'] = yes_symptoms_df['การรักษาก่อนหน้า'].apply(
        lambda x: 0 if isinstance(x, list) and 'ไม่เคย' in x[0] else 1 if isinstance(x, list) else 0
    )
    async def translate_text(text):
        translator = Translator()
        translated = await translator.translate(text, src='en', dest='th')
        return translated.text

    english_words = []
    thai_words = []
    for word in yes_symptoms_df.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)

    # ฟังชันก์หาคำที่ความหมายใกล้กันด้วยไลบรารี่ difflib
    def find_closest_match(word, choices):
        match = difflib.get_close_matches(word, choices, n=1, cutoff=0.5)
        return match[0] if match else None

    # แปลคำอังกฤษและมาเทียบกับคำไทยที่มีอยู่
    matching_pairs = {}
    for eng_word in english_words:
        translated = await translate_text(eng_word)
        closest_thai = find_closest_match(translated, thai_words)
        if closest_thai:
            matching_pairs[eng_word] = closest_thai

    for eng_col, thai_col in matching_pairs.items():
        yes_symptoms_df[thai_col] = yes_symptoms_df[thai_col].combine_first(yes_symptoms_df[eng_col])
        yes_symptoms_df.drop(columns=[eng_col], inplace=True)
    english_words = []
    thai_words = []
    for word in yes_symptoms_df.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)
    len(english_words)
    yes_symptoms_df = yes_symptoms_df.drop(english_words, axis=1)
    no_symptoms_expanded = []

    for symptoms in df['no_symptoms']:
        symptoms_dict = {}
        
        for item in symptoms:
            text = item['text']
            answers = item['answers']
            symptoms_dict[text] = answers
        
        no_symptoms_expanded.append(symptoms_dict)

    no_symptoms_df = pd.DataFrame(no_symptoms_expanded)
    english_words = []
    thai_words = []
    for word in no_symptoms_df.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)
    matching_pairs = {}
    for eng_word in english_words:
        translated = await translate_text(eng_word)
        closest_thai = find_closest_match(translated, thai_words)
        if closest_thai:
            matching_pairs[eng_word] = closest_thai
    # รวมคอลัมน์
    for eng_col, thai_col in matching_pairs.items():
        no_symptoms_df[thai_col] = no_symptoms_df[thai_col].combine_first(no_symptoms_df[eng_col])  # Fill NaN with matching column
        no_symptoms_df.drop(columns=[eng_col], inplace=True)
    # drop คอลัมน์อังกฤษที่เหลือ
    english_words = []
    thai_words = []
    for word in no_symptoms_df.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)

    no_symptoms_df = no_symptoms_df.drop(english_words, axis=1)
    # แยก column ของ idk_symptoms ออกมาเหมือนกัน
    idk_symptoms_expanded = []

    for symptoms in df['idk_symptoms']:
        symptoms_dict = {}
        
        for item in symptoms:
            text = item['text']
            answers = item['answers']
            
            symptoms_dict[text] = answers
        
        idk_symptoms_expanded.append(symptoms_dict)

    idk_symptoms_df = pd.DataFrame(idk_symptoms_expanded)
    # แยกภาษา
    english_words = []
    thai_words = []
    for word in idk_symptoms_df.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)
    #จับคู่
    matching_pairs = {}
    for eng_word in english_words:
        translated = await translate_text(eng_word)
        closest_thai = find_closest_match(translated, thai_words)
        if closest_thai:
            matching_pairs[eng_word] = closest_thai


    # รวมคอลัมน์
    for eng_col, thai_col in matching_pairs.items():
        idk_symptoms_df[thai_col] = idk_symptoms_df[thai_col].combine_first(idk_symptoms_df[eng_col])
        idk_symptoms_df.drop(columns=[eng_col], inplace=True)
    # drop คอลัมน์อังกฤษที่เหลือ
    english_words = []
    thai_words = []
    for word in idk_symptoms_df.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)
            
    idk_symptoms_df = idk_symptoms_df.drop(english_words, axis=1)
    search_term = df['search_term'].str.get_dummies(sep=', ')
    # search_term มีภาษาอังกฤษ เพราะฉะนั้นเราจะใช้วิธีจับคู่คำเหมือนเดิม
    english_words = []
    thai_words = []
    for word in search_term.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)
    len(english_words)
    matching_pairs = {}
    for eng_word in english_words:
        translated = await translate_text(eng_word)
        closest_thai = find_closest_match(translated, thai_words)
        if closest_thai:
            matching_pairs[eng_word] = closest_thai
    for eng_col, thai_col in matching_pairs.items():
        search_term[thai_col] = search_term[thai_col].combine_first(search_term[eng_col])  # Fill NaN with matching column
        search_term.drop(columns=[eng_col], inplace=True)
    english_words = []
    thai_words = []
    for word in search_term.columns:
        if re.match(r'[a-zA-Z]', word):
            english_words.append(word)
        else:
            thai_words.append(word)

    search_term = search_term.drop(english_words, axis=1)

    categories = [
        "อาการทางระบบหายใจ",  
        "อาการทางระบบทางเดินอาหาร",  
        "อาการทางระบบประสาท",  
        "อาการทางผิวหนังและภูมิแพ้",  
        "อาการปวดและการอักเสบ",  
        "อาการไข้และการติดเชื้อ", 
        "อาการทางระบบหัวใจและหลอดเลือด",
        "อาการทางระบบปัสสาวะ",  
        "อาการทางสุขภาพจิต",
        "อาการทั่วไปอื่นๆ"  
    ]
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
    # ทำการจำแนก column
    results = classifier(list(search_term.columns), candidate_labels=categories, multi_label=True)

    # นำ result ที่ได้มา process เพื่อนำค่าไปใส่ในหมวดหมู่ที่ความใกล้เคียงกับอาการ
    category_mapping = {}
    for i, col in enumerate(search_term.columns):
        matched_categories = [categories[j] for j, score in enumerate(results[i]["scores"]) if score > 0.8]
        category_mapping[col] = matched_categories if matched_categories else ["อาการทั่วไปอื่นๆ"]  # หากไม่ตรงกับหมวดหมู่ไหนเลยก็จะใช้ อาการทั่วไปอื่นๆ เป็นค่า default

    # นำค่าที่ได้มา group กันตามหมวดหมู่
    category_groups = {}
    for col, categories in category_mapping.items():
        for category in categories:
            category_groups.setdefault(category, []).append(col)

    # สร้าง Dataframe มาเพื่อเก็บค่า search term จามหมวดหมู่
    search_term_df = pd.DataFrame({cat: search_term[cols].sum(axis=1) for cat, cols in category_groups.items() if cols})


    no_symptoms_df = no_symptoms_df.add_prefix("no_symptoms_")  
    df = pd.concat([df, no_symptoms_df], axis=1)  
    df = df.drop('no_symptoms', axis=1)

    idk_symptoms_df = idk_symptoms_df.add_prefix("idk_symptoms_")
    df = pd.concat([df, idk_symptoms_df], axis=1)
    df = df.drop('idk_symptoms', axis=1)

    yes_symptoms_df = yes_symptoms_df.add_prefix("yes_symptoms_")
    df = pd.concat([df, yes_symptoms_df], axis=1)
    df = df.drop('yes_symptoms', axis=1)

    search_term_df = search_term_df.add_prefix("search_term_")
    df = pd.concat([df, search_term_df], axis=1)
    df = df.drop('search_term', axis=1)

    temp_prev = df['yes_symptoms_การรักษาก่อนหน้า'].copy()

    df.iloc[:, 2:] = df.iloc[:, 2:].applymap(
        lambda x: 1 if isinstance(x, list) else (1 if pd.notna(x) else 0)
    )

    df['yes_symptoms_การรักษาก่อนหน้า'] = temp_prev

    dummies = pd.get_dummies(df['gender'],dtype='int')

    df = pd.concat([df.drop('gender',axis=1),dummies],axis=1)

    age_scaler = joblib.load('age_scaler.pkl')
    df['age'] = age_scaler.transform(df[['age']])
    
    user_vector = process_user_input(user_data, all_columns)
    svd_model = joblib.load("svd_model.pkl")
    recommendations = recommend_symptoms(svd_model, np.array(user_vector), all_columns)
    return jsonify({"recommended_symptoms": recommendations})
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)